#### Retrieval:

In [3]:
from langchain.embeddings.base import Embeddings
import numpy as np

class SentenceTransformerEmbeddings(Embeddings): #We need to inherit from Embeddings for integrating into langchain pipeline
    def __init__(self, model):
        self.model = model

    def embed_documents(self,texts):
        return [self.model.encode(text) for text in texts]

    def embed_query(self, query):
        return self.model.encode(query)

In [4]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-mpnet-base-v2')

/opt/conda/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
sentence_embeddings = SentenceTransformerEmbeddings(embedding_model)

In [9]:

from langchain_community.vectorstores import FAISS

db = FAISS.load_local("faiss_index", sentence_embeddings, allow_dangerous_deserialization=True)


In [10]:
# Perform a query by using the retriever
retriever = db.as_retriever()  # Convert FAISS into a retriever object
query = "What is phylosophy?"
retrieved_docs = retriever.get_relevant_documents(query)

# Print out the retrieved documents
for doc in retrieved_docs:
    print(doc.page_content)
    print('\n')


/tmp/ipykernel_478/2456047785.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retrieved_docs = retriever.get_relevant_documents(query)


systematics, the field of biology that studies the diversity of life and the relationships of living things through time. Today, systematists typically treat “relatedness” solely in terms of recency of common ancestry, but this was not always the case. Pre-Darwinian taxonomists discussed the relationships of various groups and their place in the “natural system”, and while the rise of evolutionary theory allowed that one sense of relatedness was genealogical, it did not eliminate the idea of the broader notion. Debates about the role of phylogeny in classification and taxonomy were widespread (e.g., Huxley [ed.] 1940; Winsor 1995) though they began to take on a new form beginning in the late 1950s as collaborations turned into organized research programs pushing their agendas.  In his analysis of the period, David Hull (1988: ch. 5) titled one of his chapters “Systematists at War” and thus the name “The Systematics Wars” is sometimes used to describe the debates of the period. Hull (19

#### Question Answering:

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/CodeQwen1.5-7B-Chat",
    torch_dtype="auto",
    device_map="auto"
)


config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/31.7k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.71G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/CodeQwen1.5-7B-Chat")

prompt = "how to enjoy life"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.46M [00:00<?, ?B/s]

In [13]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
     ('system', "You are a helpful assistant."),
    ('user', "{input}")
])

prompt  = template.invoke({'input': 'how to enjoy life?'})

In [14]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to('cuda')

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [15]:
response

'\n\nEnjoying life can vary from person to person, but here are some tips to help you find what truly makes you happy:\n\n1. Identify your passions and interests: This will help you make time for things that you enjoy doing, which can lead to a more fulfilling life overall.\n\n2. Set goals: Having clear and specific goals can help you stay motivated and engaged. Setting goals and working towards them can create a sense of accomplishment, which can lead to greater happiness.\n\n3. Spend time with loved ones: Making time for people you care about can boost your mood and increase your overall sense of happiness.\n\n4. Practice gratitude: Focusing on the good things in your life and expressing gratitude can help you feel more content and fulfilled.\n\n5. Practice mindfulness: Mindfulness can help you become more present in your life and enjoy the present moment.\n\n6. Give back to others: Making a positive impact on others, whether by volunteering, supporting causes you care about, or bein

#### Add retrieval to the question answering:

In [39]:
from langchain_core.prompts import ChatPromptTemplate 
# chat prompt template is used for running model in conversational format

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "You're name is Dr.Phil"
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

template = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": '{input}'}
]


In [42]:
def format_template(system_prompt, context, user_input): #Custom prompt template
    template = [
        {'role':'system', 'content': system_prompt.format(context = context)},
        {'role': 'user', 'content': user_input}
    ]
    return template

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_core.runnables import Runnable
import torch



class CustomLLMRunnable(Runnable):
    def invoke(self, messages, **kwargs):
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        model_inputs = tokenizer([text], return_tensors="pt").to('cuda')
        
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512
        )
        
        # Slice generated tokens to exclude input
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        
        # Decode the response and return it
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return response

custom_llm_runnable = CustomLLMRunnable()


In [31]:
context = retriever.get_relevant_documents('what is phylosophy?')

In [44]:
def question_answer_chain(system_prompt, context, question):
    prompt = format_template(system_prompt, context, question)
    return custom_llm_runnable.invoke(prompt)

In [37]:
def rag_chain(question):
    context = retriever.get_relevant_documents(question)
    context = [document.page_content for document in context]
    return question_answer_chain(system_prompt, context, question)


In [46]:
rag_chain('Is there such a thing as true altruism, or are all acts of kindness ultimately driven by self-interest?')

"In many evolutionary theory and psychological literature, it is common to view altruism as a category of behavior characterized by acts that result from self-interested reasons or are undertaken as a form of self-protection. However, it is worth noting that there are instances where acts of kindness are driven by true altruism, which is characterized by an intention to promote the welfare of others without any personal gain.\n\nOne example of true altruism is the act of selfless service, in which an individual performs a service to another human being out of a genuine desire to help and improve their well-being. For example, a person might volunteer their time and skills at a local animal shelter to help those in need, even if it involves sacrificing their own interests or well-being.\n\nHowever, it's important to understand that true altruism is not always the best motive for a given act. In many cases, acts of kindness may arise from self-protection or self-compassion, which may lea